<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/giswqs/geemap/tree/master/examples/notebooks/usda_naip_imagery.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/geemap/blob/master/examples/notebooks/usda_naip_imagery.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/giswqs/geemap/blob/master/examples/notebooks/usda_naip_imagery.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

## Install Earth Engine API and geemap
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geemap](https://github.com/giswqs/geemap). The **geemap** Python package is built upon the [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) and [folium](https://github.com/python-visualization/folium) packages and implements several methods for interacting with Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, and `Map.centerObject()`.
The following script checks if the geemap package has been installed. If not, it will install geemap, which automatically installs its [dependencies](https://github.com/giswqs/geemap#dependencies), including earthengine-api, folium, and ipyleaflet.

**Important note**: A key difference between folium and ipyleaflet is that ipyleaflet is built upon ipywidgets and allows bidirectional communication between the front-end and the backend enabling the use of the map to capture user input, while folium is meant for displaying static data only ([source](https://blog.jupyter.org/interactive-gis-in-jupyter-with-ipyleaflet-52f9657fa7a)). Note that [Google Colab](https://colab.research.google.com/) currently does not support ipyleaflet ([source](https://github.com/googlecolab/colabtools/issues/60#issuecomment-596225619)). Therefore, if you are using geemap with Google Colab, you should use [`import geemap.eefolium`](https://github.com/giswqs/geemap/blob/master/geemap/eefolium.py). If you are using geemap with [binder](https://mybinder.org/) or a local Jupyter notebook server, you can use [`import geemap`](https://github.com/giswqs/geemap/blob/master/geemap/geemap.py), which provides more functionalities for capturing user input (e.g., mouse-clicking and moving).

In [ ]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  

In [ ]:
import geemap
from ipyleaflet import *
from ipywidgets import *
from geemap.eelib import *

Map = geemap.Map(center=(-100, 40), zoom=4)
Map.default_style = {'cursor': 'pointer'}
Map

Map.setOptions('ROADMAP')
# Load National Hydrography Dataset (NHD) 
HUC10 = ee.FeatureCollection('USGS/WBD/2017/HUC10');  # 18,487 HUC10 watersheds in the U.S.

# Add HUC layer to the map
Map.setCenter(-99.00, 47.01, 8);
Map.addLayer(ee.Image().paint(HUC10, 0, 1), {}, 'HUC-10 Watershed')   # HUC10 for the entire U.S.

label = Label("Click on the map to select a watershed")
widget_control = WidgetControl(widget=label, position='bottomright')
Map.add_control(widget_control)

layer = None

def handle_interaction(**kwargs):
    latlon = kwargs.get('coordinates')
    if kwargs.get('type') == 'click':
        Map.default_style = {'cursor': 'wait'}
        xy = ee.Geometry.Point(latlon[::-1])
        watershed = HUC10.filterBounds(xy)
        huc10_id = watershed.first().get('huc10').getInfo()
        Map.layers = Map.layers[:3]
        Map.addLayer(ee.Image().paint(watershed, 0, 2), {'palette': 'red'}, 'HUC ID: ' + huc10_id) 
        
        NAIP_images = findNAIP(watershed)
        first_image = ee.Image(NAIP_images.toList(5).get(0))
        Map.addLayer(first_image, {'bands': ['N', 'R', 'G']}, 'first image')
#         count = NAIP_images.size().getInfo()
#         for i in range(0, count):
#             image = ee.Image(NAIP_images.toList(count).get(i))
#             Map.addLayer(image, {'bands': ['N', 'R', 'G']}, str(i))
        
        Map.default_style = {'cursor': 'pointer'}

Map.on_interaction(handle_interaction)

Map

In [ ]:
Map.layers